## 自动求梯度
在深度学习中，我们经常需要对函数求梯度（gradient）。PyTorch提供的autograd包能够根据输入和前向传播过程自动构建计算图，并执行反向传播。本节将介绍如何使用autograd包来进行自动求梯度的有关操作。

### 1 概念
上一节介绍的Tensor是这个包的核心类，如果将其属性.requires_grad设置为True，它将开始追踪(track)在其上的所有操作（这样就可以利用链式法则进行梯度传播了）。完成计算后，可以调用.backward()来完成所有梯度计算。此Tensor的梯度将累积到.grad属性中。

注意在y.backward()时，如果y是标量，则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor。解释见 第2 节。

如果不想要被继续追踪，可以调用.detach()将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪，这样梯度就传不过去了。此外，还可以用with torch.no_grad()将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算可训练参数（requires_grad=True）的梯度。

Function是另外一个很重要的类。Tensor和Function互相结合就可以构建一个记录有整个计算过程的有向无环图（DAG）。每个Tensor都有一个.grad_fn属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是通过某些运算得到的，若是，则grad_fn返回一个与这些运算相关的对象，否则是None。

下面通过一些例子来理解这些概念。

### 2 Tensor
创建一个Tensor并设置requires_grad=True:

In [5]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


再做一下运算操作：

In [6]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


注意：x是直接创建的，所以它没有grad_fn, 而y是通过一个加法操作创建的，所以它有一个为的grad_fn。

像x这种直接创建的称为叶子节点，叶子节点对应的grad_fn是None。 

可通过x.isleaf 查看节点是否为叶子节点。

In [7]:
print(x.is_leaf, y.is_leaf) # True False

True False


再来点复杂度运算操作：

In [9]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


通过 ```.requires_grad_()```来用in-place的方式改变requires_grad属性：

In [10]:
a = torch.randn(2, 2) # 缺失情况下默认 requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum()
print(b.grad_fn)

False
True


### 3 梯度

```
x = torch.ones(2, 2, requires_grad=True)
y = x + 2  
z = y * y * 2
out = z.mean()
```

因为out是一个标量，所以调用backward()时不需要指定求导变量：  

**out.backward() # 等价于 out.backward(torch.tensor(1.))**

In [14]:
out.backward() # 等价于 out.backward(torch.tensor(1.))


In [15]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


令out为o,因为：  
![链式运算](https://static.sitestack.cn/projects/Dive-into-DL-PyTorch/ed2ef040ae6b39cfc3901fc9a73b47d9.png)
所以：
![链式求导结果](https://static.sitestack.cn/projects/Dive-into-DL-PyTorch/ecbafeb3b82d8fcf54e605235aeb8134.png)
所以上面的输出是正确的。
>
数学上，如果有一个函数值和自变量都为向量的函数![](https://static.sitestack.cn/projects/Dive-into-DL-PyTorch/34cb9146bb58f426ea72d9e9e1195462.png)那么 $y$ 关于 $x$ 的梯度就是一个雅可比矩阵（Jacobian matrix）:
![](https://static.sitestack.cn/projects/Dive-into-DL-PyTorch/ccaaec3f69aba45cadc627d7402f2b88.png)

而torch.autograd这个包就是用来计算一些雅克比矩阵的乘积的。例如，如果 $v$ 是一个标量函数的 $l = g(y)$ 的梯度：
![](https://static.sitestack.cn/projects/Dive-into-DL-PyTorch/77ecb58f5667aef02f2dfef302aba10d.png)


那么根据链式法则我们有 $l$ 关于 $x$ 的雅克比矩阵就为:
![](https://static.sitestack.cn/projects/Dive-into-DL-PyTorch/04ba9565ea18a514173015174dd6e2cf.png)

**注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零。**

In [16]:
# 再来反向传播一次，注意grad是累加的
out2 = x.sum()
out2.backward()
print(x.grad)
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


现在我们解释2.3.1节留下的问题，为什么在y.backward()时，如果y是标量，则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor? 简单来说就是为了避免向量（甚至更高维张量）对张量求导，而转换成标量对张量求导。举个例子，假设形状为 m x n 的矩阵 X 经过运算得到了 p x q 的矩阵 Y，Y 又经过运算得到了 s x t 的矩阵 Z。那么按照前面讲的规则，dZ/dY 应该是一个 s x t x p x q 四维张量，dY/dX 是一个 p x q x m x n的四维张量。问题来了，怎样反向传播？怎样将两个四维张量相乘？？？这要怎么乘？？？就算能解决两个四维张量怎么乘的问题，四维和三维的张量又怎么乘？导数的导数又怎么求，这一连串的问题，感觉要疯掉…… 为了避免这个问题，我们不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量。所以必要时我们要把张量通过将所有张量的元素加权求和的方式转换为标量，举个例子，假设y由自变量x计算而来，w是和y同形的张量，则y.backward(w)的含义是：先计算l = torch.sum(y * w)，则l是个标量，然后求l对自变量x的导数。   
[参考](https://zhuanlan.zhihu.com/p/29923090)

来看一些实际例子。

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

现在 z 不是一个标量，所以在调用backward时需要传入一个和z同形的权重向量进行加权求和得到一个标量。

In [ ]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

**注意: x.grad是和x同形的张量。**

再来看看中断梯度追踪的例子：

In [ ]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2
print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

可以看到，上面的y2是没有grad_fn而且y2.requires_grad=False的，而y3是有grad_fn的。如果我们将y3对x求梯度的话会是多少呢？

In [ ]:
y3.backward()
print(x.grad)

In [17]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)


现在 z 不是一个标量，所以在调用backward时需要传入一个和z同形的权重向量进行加权求和得到一个标量。

In [18]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


**注意: x.grad是和x同形的张量。**

再来看看中断梯度追踪的例子：

In [19]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2
print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


可以看到，上面的y2是没有grad_fn而且y2.requires_grad=False的，而y3是有grad_fn的。如果我们将y3对x求梯度的话会是多少呢？

In [20]:
y3.backward()
print(x.grad)

tensor(2.)
